In [1]:
from fastai.vision.all import *
import numpy as np
import pandas as pdf
import matplotlib.pyplot as plt

In [2]:
import pathlib
path = Path().absolute()

In [3]:
path.ls()

(#2) [Path('/Users/bachfischer/Documents/Study/Master - Data Science/Semester 4/Computer Vision/COMP90086-Fine-grained-localisation/src/playground.ipynb'),Path('/Users/bachfischer/Documents/Study/Master - Data Science/Semester 4/Computer Vision/COMP90086-Fine-grained-localisation/src/.ipynb_checkpoints')]

In [4]:
train_images = sorted(get_image_files(path/"../data/COMP90086_2021_Project_train/train/"))

In [5]:
print("Number of images in training dataset: ", len(train_images))

Number of images in training dataset:  7500


In [26]:
train_df = pd.read_csv("../data/COMP90086_2021_Project_train/train.csv")

In [18]:
train_df

,id,x,y
0,IMG2744_1,-9.380678,3.58271965
1,IMG2744_2,-9.380678,3.58271965
2,IMG2744_3,-9.380678,3.58271965
3,IMG2744_4,-9.380678,3.58271965
4,IMG2744_5,-9.380678,3.58271965
...,...,...,...
7495,IMG4243_1,-4.680678,35.18271965
7496,IMG4243_2,-4.680678,35.18271965
7497,IMG4243_3,-4.680678,35.18271965
7498,IMG4243_4,-4.680678,35.18271965


# Experiment 1

Source: https://www.kaggle.com/neilslab/predict-streetview-gps-using-images?select=Street+View+Images

In [ ]:
imgs = [plt.imread(fname) for fname in train_images]
#imgs = [resize(img_i, (100, 100)) for img_i in imgs]
imgs = np.array(imgs).astype(np.float32)

In [ ]:
print("Localisation data and corresponding image:\n\n", train_df.iloc[-1, :].T)
plt.imshow(imgs[-1])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(680, 490, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(2))

In [ ]:
targets = pd.concat([train_df.iloc[:,1], gps_coords.iloc[:,2]], axis=1)
inputs = imgs

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
target_scaler = scaler.fit(targets)
targets = target_scaler.transform(targets)
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, random_state=5,
                                                   test_size=0.25)

In [ ]:
history = model.fit(x_train, y_train, epochs=50, batch_size=5,
                   validation_data=(x_test, y_test))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
target_1 = targets[0].reshape(1,2)

In [ ]:
pred = model.predict(np.reshape(imgs[-1], (1,100,100,3)))
pred = target_scaler.inverse_transform(pred)
target_1 = target_scaler.inverse_transform(target_1)
print("MSE: ", (mean_squared_error(target_1, pred)))

In [ ]:
comparison = np.concatenate((pred, target_1), axis=1)
comparison = pd.DataFrame(comparison)
comparison.columns = ['Predicted x', 'Predicted y', 'True x', 'True y']
comparison = pd.concat([comparison['Predicted x'], comparison['True x'], comparison['Predicted y'], comparison['True y']], axis=1)                     
comparison.head()

## Experiment 2 (fastai)

Notes: https://fastai1.fast.ai/vision.image.html
Notes: https://docs.fast.ai/tutorial.imagenette.html

In [ ]:
dls = ImageDataLoaders.from_folder(path/"../data/COMP90086_2021_Project_train/train/", valid_pct=0.2)

In [ ]:
dls.show_batch()